# Text

In [1]:
%%capture
!pip install iblm --upgrade

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git 

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.1


In [3]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier


import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [23]:
n = 25

df = pd.read_csv(f'../data/text/text_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']
print(len(x_train))

24


In [24]:
x_train

,text
0,It breaks my heart that this movie is not appr...
1,A Murder investigation goes on back stage whil...
2,BEFORE THE DEVIL KNOWS YOU'RE DEAD starts off ...
3,You'd think you're in for some serious sightse...
4,"I saw his movie in Dallas, Texas when it came ..."
5,This documentary on schlockmeister William Cas...
6,How I got into it: When I started watching thi...
7,This was a very good film. I didn't go into it...
8,"The movie took a new angle to Gandhi's life, w..."
9,"Ok, so it may not be the award-winning ""movie ..."


In [25]:
llm_model_name = 'gpt-4'

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

### Training

In [33]:
file_path = '../models/text/'

print(f'Number of data:{len(x_train)}')
model = iblm.fit(x_train, y_train, model_name = 'text', file_path=file_path)

Number of data:24
> Start of model creating.
Tokens Used: 7301
	Prompt Tokens: 7033
	Completion Tokens: 268
Successful Requests: 1
Total Cost (USD): $0.22707


In [34]:
# Code of the model created
print(model)

import numpy as np
def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        # Analyze the text and count the positive and negative words
        positive_words = ['good', 'great', 'excellent', 'amazing', 'wonderful', 'love', 'best', 'favorite', 'enjoy', 'beautiful']
        negative_words = ['bad', 'terrible', 'awful', 'horrible', 'worst', 'hate', 'dislike', 'boring', 'ugly', 'disappointing']

        positive_count = sum([1 for word in positive_words if word in row['text'].lower()])
        negative_count = sum([1 for word in negative_words if word in row['text'].lower()])

        # Calculate the probability of the target being 1 (positive sentiment)
        total_count = positive_count + negative_count
        if total_count == 0:
            y = 0.5
        else:
            y = positive_count / total_count

        

### Prediction

In [35]:
df = pd.read_csv(f'../data/text/text_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

In [36]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [37]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.7174487508007688
Precision: 0.6831456127335267
Recall: 0.8110986547085202
F1 score: 0.7416437854658613
ROC-AUC: 0.7683282998443399
